In [ ]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import preprocess as pp
import seaborn as sns
import numpy as np
import warnings
from preprocess import merge_adatas
import scrublet as scr
from matplotlib.pyplot import rc_context
import scanpy.external as sce
import os
warnings.filterwarnings('ignore')

In [ ]:
def infer_doublets(adata, plot=True):
    scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.06)
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                      min_cells=3, 
                                                      min_gene_variability_pctl=85, 
                                                      n_prin_comps=30)
    adata.obs['doublet_scores'] = doublet_scores
    score = 0.15
    scrub.call_doublets(threshold=score)
    scrub.plot_histogram()
    if plot:
        scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
        scrub.plot_embedding('UMAP', order_points=True)
    pred_thres_doublets = doublet_scores<score
    print('Total doublets are:', np.sum(doublet_scores>=score))
    adata = adata[pred_thres_doublets,:]
    return adata

In [ ]:
file_list = os.listdir('E:\\datasets\\singlecell\\')

In [ ]:
for data_name in file_list:
    adata = sc.read_10x_mtx('E:\\datasets\\singlecell\\'+data_name+'\\filtered_feature_bc_matrix')
    print('data size beforfe doublets removal:', adata)
    adata = infer_doublets(adata, plot=False)
    plt.show()
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)
    mito_genes = adata.var_names.str.startswith('MT-')
    adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
    
    sns.histplot(adata.obs['percent_mito'])
    plt.xlim(0, 0.3)
    plt.savefig('./figures_mito/'+data_name+'percent_mito.png')
    plt.show()
    
    sns.histplot(adata.obs['n_genes'])
    plt.xlim(0, 6000)
    plt.savefig('./figures_n_genes/'+data_name+'n_genes.png')
    plt.show()
    
    adata = preprocessing(adata)
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    adata.raw = adata
    
    sc.pp.highly_variable_genes(adata, n_top_genes=3000)
    adata = adata[:, adata.var.highly_variable]
    sc.pp.scale(adata)
    
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata)
    
    sc.tl.leiden(adata, resolution=1.5)
    sc.tl.umap(adata)
    
    sc.pl.umap(adata, color=['leiden', 'doublet_scores'], wspace=0.3, ncols=2, save =data_name+ 'umap.png')
    sc.pl.matrixplot(adata, var_names={'Treg':['CTLA4', 'FOXP3','IL2RA', 'IL1R2','RTKN2','LAIR2',],
                                  'CD4T':['IL7R','CD40LG'],
                                  'CD8T':[ 'CD8A', 'CD8B'],
                                  'Cycling':[ 'RRM2','HMGN2'],
                                  'Neutrophil':['CSF3R','FCGR3B'],
                                  'Myeloid':['C1QB', 'LYZ', 'AIF1', 'RNASE1'],
                                  'Endothelial':[ 'PECAM1', 'TM4SF1'],
                                  'NK':['GNLY', 'KLRD1', 'KLRF1', 'GZMB'],
                                  'B':['MS4A1', 'BANK1', 'CD79A', 'TNFRSF13C', 'BCL11A','PAX5'],
                                  'Epithelial':['KRT18', 'KRT19', 'EPCAM', 'SOX4'],
                                  'DC3':['LAMP3', 'CCR7', 'CCL19', 'CCL22'],
                                  'pDC':['TCF4', 'TCL1A'],
                                  'Plasmacyte':['MZB1', 'IGLL5'],
                                  'Fibroblast':[ 'COL1A1', 'LUM'],
                                  'Mast':['TPSAB1','KIT']}, 
                 groupby='leiden', standard_scale='var', cmap='RdBu_r')
    #annot each leiden clusters based on markers
    adata.write(data_name+'.h5ad')